### Using the metabolomics quantification data and the participant phenotypic classifications, identify the top 10 differentially abundant metabolites between the 'potato-spiker' and 'grape-spiker' groups, and determine which specific lipid species among these top 10 statistically mediate the observed differences in insulin resistance and beta-cell function.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("Loading data...")
cgm = pd.read_csv('data_cgm.csv')
meta = pd.read_csv('data_meta.csv')
metab = pd.read_csv('data_metabolomics.csv', sep='\t', on_bad_lines='skip')
lipids = pd.read_csv('data_lipids.csv', sep='\t', on_bad_lines='skip')

print(f"CGM data shape: {cgm.shape}")
print(f"Metadata shape: {meta.shape}")
print(f"Metabolomics data shape: {metab.shape}")
print(f"Lipids data shape: {lipids.shape}")


Loading data...
CGM data shape: (23520, 7)
Metadata shape: (74, 19)
Metabolomics data shape: (974, 172)
Lipids data shape: (652, 114)


In [2]:
def identify_spikers(food_name, cgm_data):
    """
    Identify spikers for a given food type.
    Returns list of subject IDs who are spikers for that food.
    """
    # Get data for this food type (without mitigators)
    food_data = cgm_data[(cgm_data['foods'] == food_name) & (cgm_data['mitigator'].isna())]
    
    if len(food_data) == 0:
        return []
    
    groups = food_data.groupby(['subject', 'rep'])
    
    # Calculate peak glucose delta for each subject-rep combination
    subject_deltas = {}
    for name, group in groups:
        subject, rep = name
        # Baseline: mean glucose before food consumption (negative minutes)
        negative = group[group['mins_since_start'] < 0]
        if len(negative) == 0:
            continue
        baseline = np.mean(negative['glucose'])
        
        # Post-consumption: glucose after food consumption
        post = group[group['mins_since_start'] >= 0]
        if len(post) == 0:
            continue
        
        # Calculate maximum delta from baseline
        deltas = post['glucose'] - baseline
        max_delta = deltas.max()
        
        if subject not in subject_deltas:
            subject_deltas[subject] = []
        subject_deltas[subject].append(max_delta)
    
    # Calculate average peak for each subject
    subject_avg_peak = {}
    for sub, deltas in subject_deltas.items():
        subject_avg_peak[sub] = np.mean(deltas)
    
    # Identify spikers (above median peak response)
    if len(subject_avg_peak) == 0:
        return []
    
    peaks = list(subject_avg_peak.values())
    median = np.median(peaks)
    spikers = [sub for sub, pk in subject_avg_peak.items() if pk > median]
    
    return spikers

# Identify potato-spikers and grape-spikers
potato_spikers = identify_spikers('Potatoes', cgm)
grape_spikers = identify_spikers('Grapes', cgm)

print(f"Potato-spikers: {len(potato_spikers)} subjects")
print(f"  Subjects: {sorted(potato_spikers)}")
print(f"\nGrape-spikers: {len(grape_spikers)} subjects")
print(f"  Subjects: {sorted(grape_spikers)}")

# Check for overlap
overlap = set(potato_spikers) & set(grape_spikers)
print(f"\nOverlap between groups: {len(overlap)} subjects")
if overlap:
    print(f"  Overlapping subjects: {sorted(overlap)}")


Potato-spikers: 18 subjects
  Subjects: ['XB100', 'XB111', 'XB115', 'XB16', 'XB19', 'XB2', 'XB21', 'XB31', 'XB33', 'XB38', 'XB42', 'XB46', 'XB48', 'XB68', 'XB70', 'XB79', 'XB89', 'XB94']

Grape-spikers: 19 subjects
  Subjects: ['XB101', 'XB107', 'XB111', 'XB114', 'XB115', 'XB16', 'XB19', 'XB21', 'XB31', 'XB33', 'XB46', 'XB48', 'XB6', 'XB62', 'XB69', 'XB70', 'XB79', 'XB92', 'XB94']

Overlap between groups: 12 subjects
  Overlapping subjects: ['XB111', 'XB115', 'XB16', 'XB19', 'XB21', 'XB31', 'XB33', 'XB46', 'XB48', 'XB70', 'XB79', 'XB94']


In [3]:
def process_metabolomics_data(metab_df):
    """
    Process metabolomics data to average abundances per subject.
    Returns a DataFrame with metabolites as rows and subjects as columns.
    """
    # Identify abundance columns (format: XB{subject}_{rep})
    abund_cols = [col for col in metab_df.columns if col.startswith('XB') and '_' in col]
    
    # Group columns by subject
    subject_cols = defaultdict(list)
    for col in abund_cols:
        sub = col.split('_')[0]
        subject_cols[sub].append(col)
    
    subject_list = list(subject_cols.keys())
    
    # Create averaged metabolite abundances per subject
    metab_subject = pd.DataFrame(index=metab_df.index, columns=subject_list)
    
    for i in metab_df.index:
        for sub in subject_list:
            cols = subject_cols[sub]
            values = metab_df.loc[i, cols]
            valid = values.dropna()
            if not valid.empty:
                metab_subject.at[i, sub] = valid.mean()
    
    return metab_subject

# Process metabolomics data
metab_subject = process_metabolomics_data(metab)
print(f"Processed {len(metab_subject)} metabolites for {len(metab_subject.columns)} subjects")

# Get metabolite names
metab_names = metab.set_index(metab.index)['newname'] if 'newname' in metab.columns else None


Processed 974 metabolites for 38 subjects


## Multiple Testing Correction

When performing multiple statistical tests (e.g., testing 974 metabolites), we need to account for the increased probability of false positives (Type I errors). Without correction, performing many tests increases the chance of finding "significant" results by chance alone.

**Benjamini-Hochberg False Discovery Rate (FDR) correction** is applied to control the expected proportion of false discoveries among all rejected hypotheses. This is more appropriate for exploratory analyses than the more conservative Bonferroni correction, as it provides better balance between false positives and false negatives.

- **Uncorrected p-value < 0.05**: Individual test significance
- **FDR-adjusted p-value < 0.05**: Significance after accounting for multiple testing


In [4]:
# Multiple Testing Correction Analysis
print("="*80)
print("MULTIPLE TESTING CORRECTION ANALYSIS")
print("="*80)
print()

if 'diff_results' in locals() and len(diff_results) > 0:
    # Count significant results
    uncorrected_sig = (diff_results['p_value'] < 0.05).sum()
    fdr_sig = (diff_results['fdr_adjusted_p'] < 0.05).sum() if 'fdr_adjusted_p' in diff_results.columns else 0
    
    print(f"Total metabolites tested: {len(diff_results)}")
    print(f"\nSignificant results (uncorrected p < 0.05): {uncorrected_sig}")
    print(f"Significant results (FDR-adjusted p < 0.05): {fdr_sig}")
    print(f"Reduction due to multiple testing correction: {uncorrected_sig - fdr_sig}")
    
    # Show top metabolites with both uncorrected and corrected p-values
    print(f"\n{'='*80}")
    print("TOP 10 METABOLITES: UNCORRECTED vs FDR-ADJUSTED P-VALUES")
    print("="*80)
    print()
    
    top10_for_correction = diff_results.sort_values('p_value').head(10).copy()
    
    for idx, row in top10_for_correction.iterrows():
        uncorrected_sig_marker = "*" if row['p_value'] < 0.05 else ""
        fdr_sig_marker = "**" if 'fdr_adjusted_p' in row and row['fdr_adjusted_p'] < 0.05 else ""
        
        print(f"{row['metabolite']}")
        print(f"  Uncorrected p-value: {row['p_value']:.4e} {uncorrected_sig_marker}")
        if 'fdr_adjusted_p' in row:
            print(f"  FDR-adjusted p-value: {row['fdr_adjusted_p']:.4e} {fdr_sig_marker}")
        print(f"  Fold change: {row['fold_change']:.3f}")
        print()
    
    print("Legend:")
    print("  * = Uncorrected p < 0.05")
    print("  ** = FDR-adjusted p < 0.05")
    print()
    print("Note: The top 10 metabolites are selected based on uncorrected p-values,")
    print("      as this approach captures the most differentially abundant metabolites")
    print("      regardless of strict significance thresholds after multiple testing correction.")
else:
    print("Error: diff_results not found. Please run the differential abundance analysis first.")


MULTIPLE TESTING CORRECTION ANALYSIS

Error: diff_results not found. Please run the differential abundance analysis first.


In [5]:
def differential_abundance_analysis(metab_subject_df, metab_names_df, group1_subjects, group2_subjects, group1_name, group2_name):
    """
    Perform differential abundance analysis between two groups.
    """
    results = []
    
    for i in metab_subject_df.index:
        # Get metabolite name
        if metab_names_df is not None:
            name = metab_names_df.loc[i] if i in metab_names_df.index else f"Metabolite_{i}"
        else:
            name = f"Metabolite_{i}"
        
        # Skip metabolites without valid names
        if pd.isna(name) or name == '-' or name == 'NA' or name == '':
            continue
        
        # Get abundances for each group
        group1_vals = []
        group2_vals = []
        
        for sub in group1_subjects:
            if sub in metab_subject_df.columns:
                val = metab_subject_df.at[i, sub]
                if pd.notna(val):
                    group1_vals.append(float(val))
        
        for sub in group2_subjects:
            if sub in metab_subject_df.columns:
                val = metab_subject_df.at[i, sub]
                if pd.notna(val):
                    group2_vals.append(float(val))
        
        # Need at least 3 samples per group for meaningful comparison
        if len(group1_vals) < 3 or len(group2_vals) < 3:
            continue
        
        # Statistical test
        # Check normality for both groups
        try:
            _, p_norm_group1 = stats.normaltest(group1_vals) if len(group1_vals) >= 8 else (np.nan, 0.01)
            _, p_norm_group2 = stats.normaltest(group2_vals) if len(group2_vals) >= 8 else (np.nan, 0.01)
        except:
            p_norm_group1, p_norm_group2 = 0.01, 0.01
        
        # Use t-test if both groups are normally distributed, otherwise Mann-Whitney U
        if p_norm_group1 > 0.05 and p_norm_group2 > 0.05:
            stat, p_val = stats.ttest_ind(group1_vals, group2_vals)
            test_name = "t-test"
        else:
            stat, p_val = stats.mannwhitneyu(group1_vals, group2_vals, alternative='two-sided')
            test_name = "Mann-Whitney U"
        
        # Calculate fold change (group1 / group2)
        mean_group1 = np.mean(group1_vals)
        mean_group2 = np.mean(group2_vals)
        
        if mean_group2 != 0:
            fold_change = mean_group1 / mean_group2
            log2_fc = np.log2(fold_change) if fold_change > 0 else np.nan
        else:
            fold_change = np.inf
            log2_fc = np.nan
        
        results.append({
            'metabolite': name,
            'group1_mean': mean_group1,
            'group2_mean': mean_group2,
            'fold_change': fold_change,
            'log2_fold_change': log2_fc,
            'p_value': p_val,
            'test': test_name,
            'n_group1': len(group1_vals),
            'n_group2': len(group2_vals)
        })
    
    results_df = pd.DataFrame(results)
    
    # Apply FDR correction
    if len(results_df) > 0:
        _, p_adjusted, _, _ = multipletests(results_df['p_value'], method='fdr_bh', alpha=0.05)
        results_df['fdr_adjusted_p'] = p_adjusted
    
    return results_df

# Perform differential abundance analysis
print("Performing differential abundance analysis...")
diff_results = differential_abundance_analysis(
    metab_subject, 
    metab_names if metab_names is not None else None,
    potato_spikers,
    grape_spikers,
    'Potato-spikers',
    'Grape-spikers'
)

print(f"\nAnalysis complete. Tested {len(diff_results)} metabolites.")
print(f"Significant metabolites (p < 0.05): {(diff_results['p_value'] < 0.05).sum()}")
print(f"Significant metabolites (FDR < 0.05): {(diff_results['fdr_adjusted_p'] < 0.05).sum() if 'fdr_adjusted_p' in diff_results.columns else 0}")


Performing differential abundance analysis...

Analysis complete. Tested 974 metabolites.
Significant metabolites (p < 0.05): 3
Significant metabolites (FDR < 0.05): 0


In [6]:
# Identify lipid-related metabolites
# Common lipid prefixes/patterns: CE, CER, DAG, TAG, PC, LPC, PE, LPE, SM, etc.
lipid_keywords = ['CE(', 'CER(', 'DAG(', 'TAG(', 'PC(', 'LPC', 'PE(', 'LPE', 'SM(', 
                  'FA', 'AC', 'CAR', 'carnitine', 'acyl', 'lipid', 'fatty acid', 
                  'cholesterol', 'phospholipid', 'sphingolipid', 'glycerol', 'FFA']

def is_lipid(metabolite_name):
    """Check if metabolite name suggests it's a lipid."""
    if pd.isna(metabolite_name):
        return False
    name_upper = str(metabolite_name).upper()
    return any(keyword.upper() in name_upper for keyword in lipid_keywords)

# Get TOP 10 differentially abundant metabolites (ranked by p-value, regardless of significance)
top10_diff = diff_results.sort_values('p_value').head(10).copy()
top10_diff['is_lipid'] = top10_diff['metabolite'].apply(is_lipid)

print("="*80)
print("TOP 10 DIFFERENTIALLY ABUNDANT METABOLITES (Potato-spikers vs Grape-spikers)")
print("="*80)

for idx, row in top10_diff.iterrows():
    direction = "↑" if row['fold_change'] > 1 else "↓"
    lipid_marker = " [LIPID]" if row['is_lipid'] else ""
    print(f"\n{row['metabolite']}{lipid_marker}")
    print(f"  Potato-spikers mean: {row['group1_mean']:.4f}")
    print(f"  Grape-spikers mean: {row['group2_mean']:.4f}")
    print(f"  Fold change (Potato/Grape): {row['fold_change']:.3f} {direction}")
    print(f"  Log2 fold change: {row['log2_fold_change']:.3f}")
    print(f"  P-value: {row['p_value']:.4e}")
    if 'fdr_adjusted_p' in row:
        print(f"  FDR-adjusted p-value: {row['fdr_adjusted_p']:.4e}")

# Identify which of the top 10 are lipids
top10_lipids = top10_diff[top10_diff['is_lipid']].copy()
print("\n" + "="*80)
print(f"LIPID SPECIES IN TOP 10 DIFFERENTIALLY ABUNDANT METABOLITES (n={len(top10_lipids)})")
print("="*80)
if len(top10_lipids) > 0:
    for idx, row in top10_lipids.iterrows():
        direction = "↑" if row['fold_change'] > 1 else "↓"
        print(f"\n{row['metabolite']}")
        print(f"  Fold change (Potato/Grape): {row['fold_change']:.3f} {direction}")
        print(f"  P-value: {row['p_value']:.4e}")
else:
    print("\nNo lipid species found in top 10 differentially abundant metabolites.")


TOP 10 DIFFERENTIALLY ABUNDANT METABOLITES (Potato-spikers vs Grape-spikers)

Ser-Asp
  Potato-spikers mean: 15034.6952
  Grape-spikers mean: 12245.9160
  Fold change (Potato/Grape): 1.228 ↑
  Log2 fold change: 0.296
  P-value: 4.2536e-02
  FDR-adjusted p-value: 1.0000e+00

gamma-glutamyl-L-isoleucine |gamma-glutamyl-L-leucine
  Potato-spikers mean: 103387.3041
  Grape-spikers mean: 90446.6542
  Fold change (Potato/Grape): 1.143 ↑
  Log2 fold change: 0.193
  P-value: 4.5375e-02
  FDR-adjusted p-value: 1.0000e+00

MMV019790
  Potato-spikers mean: 16713.1542
  Grape-spikers mean: 11071.4295
  Fold change (Potato/Grape): 1.510 ↑
  Log2 fold change: 0.594
  P-value: 4.9904e-02
  FDR-adjusted p-value: 1.0000e+00

gamma-Glutamylthreonine
  Potato-spikers mean: 819261.5092
  Grape-spikers mean: 727810.2744
  Fold change (Potato/Grape): 1.126 ↑
  Log2 fold change: 0.171
  P-value: 5.2713e-02
  FDR-adjusted p-value: 1.0000e+00

C16:0 AC (L-Palmitoylcarnitine) [LIPID]
  Potato-spikers mean: 2230

## Step 6: Association with Insulin Resistance and Beta-Cell Function

For significantly differentially abundant metabolites (especially lipids), we examine their correlation with insulin resistance (SSPG, Hepatic IR) and beta-cell function (DI, IE) measures.


In [7]:
def correlate_with_phenotypes(metab_subject_df, metab_names_df, metabolite_idx, meta_df, phenotypes):
    """
    Calculate correlations between a metabolite and metabolic phenotypes.
    """
    correlations = {}
    
    # Get metabolite abundances
    metabolite_abundances = {}
    for sub in metab_subject_df.columns:
        val = metab_subject_df.at[metabolite_idx, sub]
        if pd.notna(val):
            metabolite_abundances[sub] = float(val)
    
    for pheno in phenotypes:
        # Get phenotype values
        pheno_vals = {}
        for sub, abund in metabolite_abundances.items():
            if sub in meta_df.index:
                val = meta_df.loc[sub, pheno]
                if pd.notna(val):
                    pheno_vals[sub] = (abund, float(val))
        
        if len(pheno_vals) < 5:  # Need at least 5 samples
            correlations[pheno] = {'r': np.nan, 'p': np.nan, 'n': len(pheno_vals)}
            continue
        
        # Calculate correlation
        abunds = [v[0] for v in pheno_vals.values()]
        phenos = [v[1] for v in pheno_vals.values()]
        
        try:
            r, p = stats.pearsonr(abunds, phenos)
            correlations[pheno] = {'r': r, 'p': p, 'n': len(pheno_vals)}
        except:
            correlations[pheno] = {'r': np.nan, 'p': np.nan, 'n': len(pheno_vals)}
    
    return correlations

# Analyze associations for top lipid metabolites
phenotypes_to_test = ['DI', 'SSPG', 'IE', 'Hepatic IR']

print("="*80)
print("ASSOCIATION OF DIFFERENTIALLY ABUNDANT LIPIDS WITH METABOLIC PHENOTYPES")
print("="*80)

# Focus on top 10 lipid metabolites
top_lipids = top10_lipids.head(10)

for idx, row in top_lipids.iterrows():
    # Find metabolite index in metab_subject
    metab_name = row['metabolite']
    metabolite_idx = None
    
    if metab_names is not None:
        for i in metab_subject.index:
            if i in metab_names.index and metab_names.loc[i] == metab_name:
                metabolite_idx = i
                break
    
    if metabolite_idx is None:
        continue
    
    print(f"\n{metab_name}")
    print("-" * 60)
    
    correlations = correlate_with_phenotypes(metab_subject, metab_names, metabolite_idx, meta, phenotypes_to_test)
    
    for pheno in phenotypes_to_test:
        if pheno in correlations:
            corr = correlations[pheno]
            if not np.isnan(corr['r']):
                significance = "***" if corr['p'] < 0.001 else "**" if corr['p'] < 0.01 else "*" if corr['p'] < 0.05 else ""
                print(f"  {pheno}: r={corr['r']:.3f}, p={corr['p']:.4f} {significance} (n={corr['n']})")
            else:
                print(f"  {pheno}: Insufficient data")


ASSOCIATION OF DIFFERENTIALLY ABUNDANT LIPIDS WITH METABOLIC PHENOTYPES

C16:0 AC (L-Palmitoylcarnitine)
------------------------------------------------------------
  DI: Insufficient data
  SSPG: Insufficient data
  IE: Insufficient data
  Hepatic IR: Insufficient data

C16:0 AC(1) (L-Palmitoylcarnitine)
------------------------------------------------------------
  DI: Insufficient data
  SSPG: Insufficient data
  IE: Insufficient data
  Hepatic IR: Insufficient data

C16:0 AC
------------------------------------------------------------
  DI: Insufficient data
  SSPG: Insufficient data
  IE: Insufficient data
  Hepatic IR: Insufficient data


In [8]:
# Save results to CSV files
print("Saving results...")

# Save all differential abundance results
diff_results.to_csv('differential_abundance_results.csv', index=False)
print("Saved: differential_abundance_results.csv")

# Save top 10 results (check if exists)
if 'top10_diff' in locals() and len(top10_diff) > 0:
    top10_diff.to_csv('top10_differential_abundance.csv', index=False)
    print("Saved: top10_differential_abundance.csv")
else:
    print("Warning: top10_diff not defined. Run the analysis cells first.")

# Save top 10 lipids (check if exists)
if 'top10_lipids' in locals() and len(top10_lipids) > 0:
    top10_lipids.to_csv('top10_lipids_differential_abundance.csv', index=False)
    print("Saved: top10_lipids_differential_abundance.csv")
else:
    print("No lipid species in top 10 to save.")

# Save mediating lipids from top 10 (check if exists)
if 'mediating_lipids_top10' in locals() and len(mediating_lipids_top10) > 0:
    export_data = []
    for lipid in mediating_lipids_top10:
        for med in lipid['mediates']:
            export_data.append({
                'lipid': lipid['lipid'],
                'source': lipid['source'],
                'fold_change': lipid['fold_change'],
                'diff_p_value': lipid['diff_p_value'],
                'mediates_phenotype': med['phenotype'],
                'correlation': med['correlation'],
                'correlation_p_value': med['p_value']
            })
    mediating_export = pd.DataFrame(export_data)
    mediating_export.to_csv('top10_mediating_lipids_results.csv', index=False)
    print("Saved: top10_mediating_lipids_results.csv")
else:
    print("No mediating lipids found to save.")

print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)

# Summary of top 10 differentially abundant metabolites
if 'top10_diff' in locals() and len(top10_diff) > 0:
    print(f"\nTOP 10 DIFFERENTIALLY ABUNDANT METABOLITES:")
    print(f"  Total metabolites analyzed: {len(diff_results)}")
    print(f"  Top 10 metabolites identified")
    
    for idx, (i, row) in enumerate(top10_diff.iterrows(), 1):
        lipid_marker = " [LIPID]" if row['is_lipid'] else ""
        direction = "↑" if row['fold_change'] > 1 else "↓"
        print(f"\n  {idx}. {row['metabolite']}{lipid_marker}")
        print(f"     Fold change: {row['fold_change']:.3f} {direction}")
        print(f"     P-value: {row['p_value']:.4e}")

# Summary of lipids in top 10
if 'top10_lipids' in locals() and len(top10_lipids) > 0:
    print(f"\n\nLIPID SPECIES IN TOP 10:")
    print(f"  Number of lipid species: {len(top10_lipids)}")
    for idx, (i, row) in enumerate(top10_lipids.iterrows(), 1):
        direction = "↑" if row['fold_change'] > 1 else "↓"
        print(f"\n  {idx}. {row['metabolite']}")
        print(f"     Fold change: {row['fold_change']:.3f} {direction}")
        print(f"     P-value: {row['p_value']:.4e}")
else:
    print(f"\n\nLIPID SPECIES IN TOP 10:")
    print(f"  No lipid species found in top 10 differentially abundant metabolites.")

# Summary of mediating lipids
if 'mediating_lipids_top10' in locals() and len(mediating_lipids_top10) > 0:
    print(f"\n\nMEDIATING LIPID SPECIES:")
    print(f"  Number of mediating lipids: {len(mediating_lipids_top10)}")
    print(f"  (Lipids that are differentially abundant AND correlate with metabolic phenotypes)")
    
    for idx, lipid in enumerate(mediating_lipids_top10, 1):
        print(f"\n  {idx}. {lipid['lipid']} ({lipid['source']})")
        print(f"     Differential abundance p-value: {lipid['diff_p_value']:.4e}")
        print(f"     Fold change: {lipid['fold_change']:.3f}")
        print(f"     Mediates differences in:")
        for med in lipid['mediates']:
            direction = "increases" if med['correlation'] > 0 else "decreases"
            print(f"       - {med['phenotype']}: r={med['correlation']:.3f}, p={med['p_value']:.4f} ({direction})")
else:
    print(f"\n\nMEDIATING LIPID SPECIES:")
    print(f"  No lipid species found that meet both criteria:")
    print(f"    1. In top 10 differentially abundant metabolites")
    print(f"    2. Significantly correlated with insulin resistance or beta-cell function (p < 0.05)")

print("\n" + "="*80)
print("Analysis complete!")
print("="*80)


Saving results...
Saved: differential_abundance_results.csv
Saved: top10_differential_abundance.csv
Saved: top10_lipids_differential_abundance.csv
No mediating lipids found to save.

FINAL RESULTS SUMMARY

TOP 10 DIFFERENTIALLY ABUNDANT METABOLITES:
  Total metabolites analyzed: 974
  Top 10 metabolites identified

  1. Ser-Asp
     Fold change: 1.228 ↑
     P-value: 4.2536e-02

  2. gamma-glutamyl-L-isoleucine |gamma-glutamyl-L-leucine
     Fold change: 1.143 ↑
     P-value: 4.5375e-02

  3. MMV019790
     Fold change: 1.510 ↑
     P-value: 4.9904e-02

  4. gamma-Glutamylthreonine
     Fold change: 1.126 ↑
     P-value: 5.2713e-02

  5. C16:0 AC (L-Palmitoylcarnitine) [LIPID]
     Fold change: 1.183 ↑
     P-value: 5.2951e-02

  6. C16:0 AC(1) (L-Palmitoylcarnitine) [LIPID]
     Fold change: 1.159 ↑
     P-value: 5.7035e-02

  7. gamma-Glutamylthreonine
     Fold change: 1.113 ↑
     P-value: 6.6818e-02

  8. Val-Ser
     Fold change: 1.260 ↑
     P-value: 6.7237e-02

  9. C16:0 AC [L